In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from sentence_transformers import SentenceTransformer
from qdrant_client.models import PointStruct
import numpy as np
from qdrant_client import QdrantClient
load_dotenv()

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
chat = ChatOpenAI(
    model='gpt-4o-mini',
    openai_api_key=os.environ['OPENAI_API_KEY'],
    temperature=0.7
)

In [3]:
messages = [
    SystemMessage(content="You are a helpful warm motivational assistant. I want your responses like a informal talk"),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to know how to be happy.")
]

In [4]:
embeddings_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# Conectamos con Qdrant Cloud
url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_KEY")
qdrant = QdrantClient(url=url, api_key=api_key)

C:\Users\Usuario1\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def custom_prompt_Content_Stopwords(query: str):
    # Generamos los embeddings de la consulta
    query_embedding = embeddings_model.encode([query])[0].tolist() 

    # Realizamos la búsqueda en la colección de Qdrant. 
    results = qdrant.search(
        collection_name="OnlyContent_withStopwords",
        query_vector=query_embedding,
        limit=3  
    )
    
    # Se genera el contexto a partir de los resultados de Qdrant.
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])
    
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt

In [6]:
def custom_prompt_Content_NOStopwords(query: str):
    
    query_embedding = embeddings_model.encode([query])[0].tolist()  

    results = qdrant.search(
        collection_name="OnlyContent_withoutStopwords",
        query_vector=query_embedding,
        limit=3 
    )
    
    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



In [7]:
def custom_prompt_Weighted_Stopwords(query: str):

    query_embedding = embeddings_model.encode([query])[0].tolist() 
    
    results = qdrant.search(
        collection_name="Weighted_withStopwords",
        query_vector=query_embedding,
        limit=3  
    )

    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])

    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



In [8]:
def custom_prompt_Weighted_NOStopwords(query: str):

    query_embedding = embeddings_model.encode([query])[0].tolist()  

    results = qdrant.search(
        collection_name="Weighted_withoutStopwords",
        query_vector=query_embedding,
        limit=3 
    )

    source_knowledge = "\n".join([result.payload.get("text", "No text available") for result in results])
    
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augment_prompt



## PREGUNTAS REALIZADAS

In [10]:
query = "¿Cuál es la capital de francia?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡La capital de Francia es París! Es una ciudad hermosa, llena de historia, cultura y esos cafés encantadores. ¿Has estado allí o te gustaría visitarla?


In [11]:
query = "¿Cuántos kilómetros hay de la Tierra a la Luna?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Hey! Pues, la distancia promedio de la Tierra a la Luna es de unos 384,400 kilómetros. Es un viaje bastante largo, pero ¡imagínate lo increíble que debe ser estar ahí arriba! Si tienes más preguntas o quieres saber algo más, ¡aquí estoy!


In [12]:
query = "Explícame cómo resolver una ecuación cuadrática"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Resolver una ecuación cuadrática es más fácil de lo que parece. Aquí te dejo una forma sencilla de hacerlo:

1. **Identifica la forma de la ecuación**: Generalmente, una ecuación cuadrática se presenta como \( ax^2 + bx + c = 0 \), donde \(a\), \(b\) y \(c\) son números reales y \(a \neq 0\).

2. **Usa la fórmula cuadrática**: La forma más común de resolverla es usando la fórmula cuadrática:
   \[
   x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
   \]
   Aquí, el símbolo \( \pm \) significa que tendrás dos soluciones: una con \(+\) y otra con \(-\).

3. **Calcula el discriminante**: Primero, calcula el valor de \( b^2 - 4ac \):
   - Si el resultado es positivo, tendrás dos soluciones reales distintas.
   - Si es cero, tendrás una solución real (o dos iguales).
   - Si es negativo, no tendrás soluciones reales, pero sí soluciones complejas.

4. **Sustituye y simplifica**: Una vez que tengas el discriminante, sustituye \( a \), \( b \) y el discriminante en la fórmula cuadrática y simpl

In [13]:
query = "¿Puedes contarme un chiste?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Aquí va uno: 

¿Cuál es el colmo de un electricista? 

¡No encontrar su corriente de trabajo! 

Espero que te haya sacado una sonrisa. Si quieres escuchar más, ¡dímelo!


In [14]:
query = "Dame la receta de un pastel de chocolate"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Hacer un pastel de chocolate es muy divertido y delicioso. Aquí te dejo una receta básica:

### Ingredientes:
- **1 ¾ tazas** de harina para todo uso
- **¾ taza** de cacao en polvo sin azúcar
- **2 tazas** de azúcar
- **1 ½ cucharaditas** de polvo de hornear
- **1 ½ cucharaditas** de bicarbonato de sodio
- **1 cucharadita** de sal
- **2 huevos**
- **1 taza** de leche
- **½ taza** de aceite vegetal
- **2 cucharaditas** de extracto de vainilla
- **1 taza** de agua hirviendo

### Instrucciones:
1. **Precalienta el horno** a 350°F (175°C). Engrasa y enharina dos moldes para pastel de 9 pulgadas.

2. En un tazón grande, **mezcla la harina, el cacao en polvo, el azúcar, el polvo de hornear, el bicarbonato de sodio y la sal**. Asegúrate de que todo esté bien combinado.

3. Agrega los **huevos, la leche, el aceite y la vainilla** a la mezcla seca. Mezcla bien hasta que esté suave.

4. Luego, **incorpora el agua hirviendo**. La mezcla será bastante líquida, ¡eso es normal!

5. Divide la

In [15]:
query = "¿Quién ganó la última Copa del Mundo de Fútbol?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

La última Copa del Mundo de Fútbol se celebró en 2022 y fue ganada por Argentina. Fue un torneo emocionante, y Argentina se coronó campeona tras vencer a Francia en una final dramática. ¡Un gran logro para el fútbol argentino!


In [16]:
query = "¿Qué opinas de la última película de Marvel?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Ah, las películas de Marvel! Siempre hay tanto debate y emoción en torno a ellas, ¿no? Si has visto la última, probablemente te hayas dado cuenta de que suelen mezclar acción trepidante con algo de humor y, claro, una buena dosis de personajes carismáticos. 

Algunas personas adoran cómo Marvel ha logrado crear un universo interconectado donde cada película cuenta una parte de una historia más grande. Otros, sin embargo, sienten que algunas películas pueden ser un poco predecibles o que siguen una fórmula similar. Pero, al final del día, lo que importa es si la película te entretuvo y te hizo sentir algo, ya sea emoción, risa o incluso nostalgia por personajes que has llegado a querer.

Así que, ¿qué opinas tú? ¿Te gustó la última película de Marvel? ¡Siempre es interesante escuchar diferentes perspectivas!


In [17]:
query = "¿Cuál es tu libro favorito y por qué?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Oh, esa es una gran pregunta! No tengo un libro favorito, pero puedo decirte que hay muchos que han dejado una huella profunda en la forma en que veo el mundo. Por ejemplo, "Cien años de soledad" de Gabriel García Márquez es un libro fascinante porque te sumerge en un mundo mágico y realista al mismo tiempo. La forma en que García Márquez entrelaza la historia de la familia Buendía con la historia de Macondo es simplemente cautivadora. 

Además, "El alquimista" de Paulo Coelho es otro que me gusta mucho, ya que habla sobre la búsqueda de los sueños y el camino que uno debe seguir para encontrarlos. La idea de que el universo conspira a favor de aquellos que persiguen sus pasiones es muy inspiradora. 

Entonces, si me preguntas, creo que cualquier libro que te haga reflexionar sobre la vida, el amor, la creatividad o la búsqueda de tus sueños puede ser un favorito en cualquier momento. ¿Tú tienes un libro favorito?


In [18]:
query = "Dime una frase inspiradora de un líder mundial"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Una frase inspiradora de un líder mundial es: "La mejor manera de predecir el futuro es crearlo." - Peter Drucker. Esta frase nos recuerda que somos capaces de influir y dar forma a nuestro futuro a través de nuestras acciones y decisiones. ¡Así que a crear ese futuro que deseas!


In [19]:
query = "¿Qué es la inteligencia artificial?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

La inteligencia artificial (IA) es un campo de estudio dentro de la informática que busca crear sistemas y programas capaces de realizar tareas que normalmente requieren inteligencia humana. Esto incluye habilidades como el aprendizaje, el razonamiento, la percepción y la comprensión del lenguaje. En otras palabras, la IA intenta emular funciones cognitivas humanas mediante algoritmos y modelos matemáticos. Existen diferentes tipos de IA, desde sistemas que pueden llevar a cabo tareas específicas, como reconocimiento de voz o imágenes, hasta sistemas más avanzados que pueden aprender y adaptarse a nuevas situaciones. En resumen, la inteligencia artificial busca hacer que las máquinas piensen y actúen de manera similar a los humanos, facilitando así una amplia gama de aplicaciones en nuestra vida diaria.


In [20]:
query = "Dame tres razones para comprar un coche eléctrico"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Aquí van tres razones para comprar un coche eléctrico:

1. **Ecológicos**: Los coches eléctricos producen cero emisiones durante su funcionamiento, lo que significa que ayudan a reducir la contaminación del aire y la huella de carbono. Al optar por un coche eléctrico, estás contribuyendo a un futuro más limpio y sostenible.

2. **Ahorro en costos**: Aunque la inversión inicial puede ser mayor, los coches eléctricos suelen tener menores costos de mantenimiento y operación. La electricidad para cargar un coche eléctrico suele ser más barata que la gasolina, y también son menos costosos de mantener a largo plazo, ya que tienen menos piezas móviles que un motor de combustión interna.

3. **Incentivos y beneficios**: Muchos gobiernos ofrecen incentivos para la compra de coches eléctricos, como descuentos fiscales, subsidios o acceso a carriles especiales. Además, algunos lugares también permiten la carga gratuita en estaciones públicas, lo que puede ser un gran ahorro.

Así que, si 

In [21]:
query = "¿Cuál es el significado de la vida según la filosofía?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

El significado de la vida según la filosofía es un tema amplio y diverso, ya que hay muchas corrientes y pensadores que han abordado esta pregunta desde diferentes perspectivas. En general, la filosofía invita a la reflexión profunda sobre la existencia, la moralidad, y el propósito personal.

1. **Búsqueda de la Verdad y el Conocimiento**: Muchos filósofos, como Sócrates, han argumentado que el significado de la vida está en la búsqueda del conocimiento y la verdad. La idea es que al cuestionar y entender el mundo que nos rodea, encontramos un propósito más profundo.

2. **Relaciones Humanas**: A lo largo de la historia, filósofos como Aristóteles han enfatizado la importancia de las relaciones humanas y la comunidad. La felicidad y la realización personal se logran a través de conexiones significativas con otros, lo que se alinea con la idea de que el amor y la amistad son esenciales para una vida plena.

3. **Aceptación de la Impermanencia**: Otras corrientes filosóficas, como el es

In [22]:
query = "Explícame cómo funciona el sistema inmunológico"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Aunque no tengo un contexto específico sobre el sistema inmunológico, puedo darte una explicación general de cómo funciona.

El sistema inmunológico es la defensa natural del cuerpo contra infecciones y enfermedades. Está compuesto por una red de células, tejidos y órganos que trabajan juntos para proteger el organismo.

1. **Barreras Físicas**: La primera línea de defensa incluye barreras físicas como la piel y las membranas mucosas. Estas actúan como un escudo, evitando que patógenos (como bacterias y virus) entren al cuerpo.

2. **Respuesta Innata**: Si los patógenos logran atravesar estas barreras, el sistema inmunológico innato responde rápidamente. Esto incluye células como los macrófagos y neutrófilos, que atacan y destruyen a los invasores. También se producen sustancias químicas que ayudan a controlar la inflamación y alertar a otras partes del sistema inmunológico.

3. **Respuesta Adaptativa**: Si la respuesta innata no es suficiente, se activa el sistema inmunológico

In [23]:
query = "¿Quién escribió 'Cien años de soledad'?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

"Cien años de soledad" fue escrito por Gabriel García Márquez. Es una de las obras más importantes de la literatura latinoamericana y se considera un clásico del realismo mágico. Si quieres saber más sobre el libro o su autor, ¡pregunta!


In [24]:
query = "¿Cuál es la ecuación de la teoría de la relatividad?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

La ecuación más famosa de la teoría de la relatividad de Einstein es \(E=mc^2\), que establece que la energía (E) es igual a la masa (m) multiplicada por la velocidad de la luz (c) al cuadrado. Esta fórmula implica que la masa y la energía son intercambiables y que pequeñas cantidades de masa pueden convertirse en grandes cantidades de energía. ¡Es un concepto fascinante que revolucionó nuestra comprensión del universo!


In [25]:
query = "¿Puedes resumir la trama de 'El Señor de los Anillos'?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

Claro, aquí va un resumen de la trama de "El Señor de los Anillos":

La historia sigue a un hobbit llamado Frodo Bolsón, quien hereda un anillo mágico de su tío Bilbo. Sin embargo, este anillo es en realidad el Anillo Único, forjado por el oscuro señor Sauron para controlar a todos los demás anillos de poder. Sauron busca recuperar el anillo para dominar la Tierra Media. 

Frodo, consciente del peligro que representa el anillo, se embarca en una misión para destruirlo. Para ello, se une a un grupo diverso conocido como la Comunidad del Anillo, que incluye a sus amigos hobbits Sam, Merry y Pippin, el mago Gandalf, el guerrero Aragorn, el elfo Legolas, el enano Gimli y el hombre Boromir. 

A lo largo de su viaje, enfrentan múltiples desafíos, incluyendo la traición, la lucha contra criaturas malignas y la tentación del propio anillo. Frodo y Sam se aventuran hacia el Monte del Destino, el único lugar donde el anillo puede ser destruido. Mientras tanto, Aragorn, Legolas y Gimli luchan par

In [26]:
query = "¿Cómo se prepara un buen café?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Hacer un buen café puede ser todo un arte, pero aquí te dejo una receta sencilla que te ayudará a disfrutar de una deliciosa taza:

1. **Elige el café**: Opta por granos de café de buena calidad. Puedes elegir entre diferentes tipos de café, como arábica o robusta, dependiendo de tu preferencia. Asegúrate de que sean frescos.

2. **Muele los granos**: Si tienes granos enteros, muele suficiente café para una o dos tazas justo antes de preparar el café. Lo ideal es un molido medio para la mayoría de los métodos de preparación.

3. **Proporción**: Usa aproximadamente 1 a 2 cucharadas de café molido por cada 6 onzas de agua. Ajusta según tu gusto personal.

4. **Agua**: Utiliza agua filtrada o mineral para mejorar el sabor. Calienta el agua hasta que esté justo antes de hervir (alrededor de 200 °F o 93 °C).

5. **Método de preparación**: Hay muchos métodos para hacer café, como la prensa francesa, la cafetera de goteo, la cafetera italiana (moka) o el método pour-over. Elige el que

In [27]:
query = "¿Qué es la fotosíntesis?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

La fotosíntesis es el proceso mediante el cual las plantas, algas y algunas bacterias convierten la luz solar en energía química. A través de este proceso, utilizan dióxido de carbono del aire y agua del suelo para producir glucosa y oxígeno. La glucosa sirve como fuente de energía y como un componente esencial para el crecimiento de la planta. En términos simples, la fotosíntesis permite a las plantas "comer" luz y crear su propio alimento, lo que es fundamental para la vida en la Tierra, ya que también produce el oxígeno que respiramos.


In [28]:
query = "¿Cuál es tu canción favorita y por qué?"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Aunque no tengo preferencias personales, puedo ayudarte a pensar en cómo podrías responder esa pregunta. Podrías decir algo como:

"Una de mis canciones favoritas es 'Imagine' de John Lennon. Me encanta porque transmite un mensaje poderoso de paz y unidad en un mundo lleno de divisiones. La melodía es hermosa y me hace reflexionar sobre la posibilidad de un mundo mejor. Además, las letras me inspiran a soñar y a trabajar por un futuro más armonioso."

Así que, si tienes una canción favorita, podrías compartir algo similar sobre por qué te gusta. ¡Espero que esto te ayude!


In [29]:
query = "Dame un consejo sobre cómo superar el miedo a hablar en público"
prompt = HumanMessage(
   content=custom_prompt_Content_Stopwords(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

¡Claro! Aquí va un consejo sobre cómo superar el miedo a hablar en público: 

Primero, es importante aceptar que es completamente normal sentirse nervioso. La clave está en cambiar tu mentalidad hacia el miedo. En lugar de pensar en lo que podría salir mal, enfócate en el mensaje que deseas transmitir y en cómo puedes conectar con tu audiencia. 

Además, puedes practicar la técnica del "sí, y..." que se menciona en uno de los contextos. Esto te ayudará a mantener la mente abierta y receptiva. Por ejemplo, si te sientes nervioso, puedes decirte a ti mismo: "Sí, estoy nervioso, y eso está bien porque estoy aquí para compartir algo valioso". 

Finalmente, asegúrate de prepararte bien y practicar repetidamente. Cuanto más familiarizado estés con tu material, más confianza tendrás. Y recuerda, incluso los mejores oradores han tenido miedo en algún momento, pero han aprendido a manejarlo y a usarlo a su favor. ¡Así que adelante, tú puedes hacerlo!
